# Validate Integration With MAR Dataset

Use this notebook to validate a manually-downloaded dataset before integrating it into the existing framework, without framework code changes.

In [ ]:
# Cell 1: Setup
from pathlib import Path
import numpy as np
import pandas as pd
import yaml

# --- user inputs ---
SOURCE_PATH = Path(r"data\\incoming\\manual_dataset.xlsx")  # .csv or .xlsx
SOURCE_SHEET = 0  # for Excel only
TARGET_BUSINESS_DATE = "2026-01-31"  # optional focus date
MODEL_CATALOG_PATH = Path(r"configs\\model_catalog.yaml")
STAGING_ROOT = Path(r"data\\staging_manual_validation")

# thresholds
SHIFT_ABS_TOL = 1e-8
SHIFT_REL_TOL = 1e-4
MIN_ROWS = 100  # sanity floor

In [ ]:
# Cell 2: Load source
if SOURCE_PATH.suffix.lower() in [".xlsx", ".xls"]:
    src = pd.read_excel(SOURCE_PATH, sheet_name=SOURCE_SHEET)
else:
    src = pd.read_csv(SOURCE_PATH)

print(f"rows={len(src):,}, cols={len(src.columns)}")
print(src.columns.tolist())

In [ ]:
# Cell 3: Normalize/Map source columns -> canonical names
def find_col(df, candidates):
    m = {c.lower().strip(): c for c in df.columns}
    for cand in candidates:
        key = cand.lower().strip()
        if key in m:
            return m[key]
    return None

col_map = {
    "business_date_raw": find_col(src, ["business dates", "businessDate", "business_date"]),
    "risk_factor_id_raw": find_col(src, ["driverName", "risk_factor_id"]),
    "close_date_raw": find_col(src, ["closeDate", "close_date", "date"]),
    "close_value_raw": find_col(src, ["closeValue", "close_value", "value"]),
    "prev_close_date_raw": find_col(src, ["prevCLoseDate", "prevCloseDate", "prev_close_date"]),
    "prev_close_value_raw": find_col(src, ["preCloseValue", "prevCloseValue", "prev_close_value"]),
    "shift_raw": find_col(src, ["shifts", "shift", "return"]),
    "asset_class_raw": find_col(src, ["assetClass", "asset_class", "rf_level1"]),
    "ccy_raw": find_col(src, ["ccy", "currency", "rf_level2"]),
    "rf_level1_raw": find_col(src, ["rf_level1"]),
    "rf_level2_raw": find_col(src, ["rf_level2"]),
    "rf_level3_raw": find_col(src, ["rf_level3"]),
    "rf_level4_raw": find_col(src, ["rf_level4"]),
    "rf_level5_raw": find_col(src, ["rf_level5"]),
}
col_map

In [ ]:
# Cell 4: Validation framework
results = []

def check(name, ok, severity="ERROR", detail=""):
    results.append({"check": name, "ok": bool(ok), "severity": severity, "detail": str(detail)})
    status = "PASS" if ok else f"{severity}:FAIL"
    print(f"[{status}] {name} {('- ' + str(detail)) if detail else ''}")

In [ ]:
# Cell 5: Check 1 - required columns
required_raw = ["business_date_raw", "risk_factor_id_raw", "close_date_raw", "close_value_raw",
                "rf_level1_raw", "rf_level2_raw", "rf_level3_raw", "rf_level4_raw", "rf_level5_raw"]
missing_required = [k for k in required_raw if col_map[k] is None]
check("Required source columns present", len(missing_required) == 0, detail=f"missing={missing_required}")
check("Minimum row count", len(src) >= MIN_ROWS, severity="WARN", detail=f"rows={len(src):,}")

In [ ]:
# Cell 6: Build canonical dataframe
df = pd.DataFrame()
df["business_date"] = src[col_map["business_date_raw"]] if col_map["business_date_raw"] else pd.NaT
df["risk_factor_id"] = src[col_map["risk_factor_id_raw"]] if col_map["risk_factor_id_raw"] else None
df["close_date"] = src[col_map["close_date_raw"]] if col_map["close_date_raw"] else pd.NaT
df["close_value"] = src[col_map["close_value_raw"]] if col_map["close_value_raw"] else np.nan
df["prev_close_date"] = src[col_map["prev_close_date_raw"]] if col_map["prev_close_date_raw"] else pd.NaT
df["prev_close_value"] = src[col_map["prev_close_value_raw"]] if col_map["prev_close_value_raw"] else np.nan
df["shift_src"] = src[col_map["shift_raw"]] if col_map["shift_raw"] else np.nan

df["assetClass"] = src[col_map["asset_class_raw"]] if col_map["asset_class_raw"] else None
df["ccy"] = src[col_map["ccy_raw"]] if col_map["ccy_raw"] else None
df["rf_level1"] = src[col_map["rf_level1_raw"]] if col_map["rf_level1_raw"] else None
df["rf_level2"] = src[col_map["rf_level2_raw"]] if col_map["rf_level2_raw"] else None
df["rf_level3"] = src[col_map["rf_level3_raw"]] if col_map["rf_level3_raw"] else None
df["rf_level4"] = src[col_map["rf_level4_raw"]] if col_map["rf_level4_raw"] else None
df["rf_level5"] = src[col_map["rf_level5_raw"]] if col_map["rf_level5_raw"] else None

# coerce
df["business_date"] = pd.to_datetime(df["business_date"], errors="coerce").dt.normalize()
df["close_date"] = pd.to_datetime(df["close_date"], errors="coerce")
df["prev_close_date"] = pd.to_datetime(df["prev_close_date"], errors="coerce")
df["close_value"] = pd.to_numeric(df["close_value"], errors="coerce")
df["prev_close_value"] = pd.to_numeric(df["prev_close_value"], errors="coerce")
df["shift_src"] = pd.to_numeric(df["shift_src"], errors="coerce")
df["risk_factor_id"] = df["risk_factor_id"].astype("string")

print(df.head(3))

In [ ]:
# Cell 7: Check 2/3 - parse + null checks
check("business_date parse success", df["business_date"].notna().mean() > 0.99, detail=f"success={df['business_date'].notna().mean():.2%}")
check("close_date parse success", df["close_date"].notna().mean() > 0.99, detail=f"success={df['close_date'].notna().mean():.2%}")
check("close_value numeric success", df["close_value"].notna().mean() > 0.99, detail=f"success={df['close_value'].notna().mean():.2%}")

critical_nulls = {
    "business_date": int(df["business_date"].isna().sum()),
    "risk_factor_id": int(df["risk_factor_id"].isna().sum()),
    "close_date": int(df["close_date"].isna().sum()),
    "close_value": int(df["close_value"].isna().sum()),
}
check("Critical null counts", sum(critical_nulls.values()) == 0, detail=critical_nulls)

In [ ]:
# Cell 8: Build timeseries_raw output + Check 4 duplicates
cur = df[["business_date", "risk_factor_id", "close_date", "close_value"]].rename(columns={"close_date":"date","close_value":"value"})
prev = df[["business_date", "risk_factor_id", "prev_close_date", "prev_close_value"]].rename(columns={"prev_close_date":"date","prev_close_value":"value"})
ts = pd.concat([cur, prev], ignore_index=True)
ts = ts.dropna(subset=["business_date", "risk_factor_id", "date", "value"]).copy()

dup_cnt = int(ts.duplicated(subset=["business_date", "risk_factor_id", "date"]).sum())
check("No duplicate (business_date, risk_factor_id, date)", dup_cnt == 0, detail=f"duplicates={dup_cnt}")

if dup_cnt > 0:
    ts = ts.sort_values(["business_date", "risk_factor_id", "date"]).drop_duplicates(
        subset=["business_date", "risk_factor_id", "date"], keep="last"
    )

In [ ]:
# Cell 9: Check 5 temporal logic
mask_prev = df["prev_close_date"].notna() & df["close_date"].notna()
bad_temporal = int((df.loc[mask_prev, "prev_close_date"] >= df.loc[mask_prev, "close_date"]).sum())
check("prev_close_date < close_date", bad_temporal == 0, detail=f"violations={bad_temporal}")

In [ ]:
# Cell 10: Check 6 shift consistency (absolute + relative candidates)
has_prev = df["prev_close_value"].notna() & df["close_value"].notna()
calc_abs = df["close_value"] - df["prev_close_value"]
calc_rel = np.where(df["prev_close_value"].abs() > 1e-12, calc_abs / df["prev_close_value"], np.nan)

if df["shift_src"].notna().any():
    valid = has_prev & df["shift_src"].notna()
    diff_abs = (df.loc[valid, "shift_src"] - calc_abs.loc[valid]).abs()
    diff_rel = (df.loc[valid, "shift_src"] - calc_rel.loc[valid]).abs()
    match_abs = (diff_abs <= SHIFT_ABS_TOL) | (diff_abs <= SHIFT_REL_TOL * df.loc[valid, "shift_src"].abs().clip(lower=1e-12))
    match_rel = (diff_rel <= SHIFT_ABS_TOL) | (diff_rel <= SHIFT_REL_TOL * df.loc[valid, "shift_src"].abs().clip(lower=1e-12))
    best_match = np.maximum(match_abs.astype(int), match_rel.astype(int)).mean() if valid.any() else 1.0
    check("Shift consistency vs source", best_match >= 0.95, severity="WARN", detail=f"match_rate={best_match:.2%}")
else:
    check("Shift consistency vs source", True, severity="INFO", detail="No source shifts column populated")

In [ ]:
# Cell 11: Build risk_factors metadata + Check 7 hierarchy integrity
meta = df[["risk_factor_id","assetClass","ccy","rf_level1","rf_level2","rf_level3","rf_level4","rf_level5"]].drop_duplicates("risk_factor_id").copy()
meta["rf_level1"] = meta["rf_level1"].fillna(meta["assetClass"])
meta["rf_level2"] = meta["rf_level2"].fillna(meta["ccy"])
for c in ["rf_level1","rf_level2","rf_level3","rf_level4","rf_level5"]:
    meta[c] = meta[c].fillna("NA").astype(str).str.strip().replace("", "NA")

missing_hier = int((meta[["rf_level1","rf_level2","rf_level3","rf_level4","rf_level5"]] == "NA").any(axis=1).sum())
dup_meta = int(meta.duplicated("risk_factor_id").sum())

check("Metadata unique risk_factor_id", dup_meta == 0, detail=f"duplicates={dup_meta}")
check("Hierarchy completeness (no NA across rf levels)", missing_hier == 0, severity="WARN", detail=f"rows_with_NA_levels={missing_hier}")

In [ ]:
# Cell 12: Check 8 snapshot integrity (factor count by business_date)
snapshot = ts.groupby(ts["business_date"].dt.strftime("%Y-%m-%d"))["risk_factor_id"].nunique().sort_index()
check("Snapshot integrity has >=1 business_date", len(snapshot) > 0, detail=f"business_dates={len(snapshot)}")
print(snapshot.tail(10))

if TARGET_BUSINESS_DATE:
    present = TARGET_BUSINESS_DATE in snapshot.index
    check(f"Target business date exists ({TARGET_BUSINESS_DATE})", present)

In [ ]:
# Cell 13: Check 9 config compatibility
cfg = yaml.safe_load(MODEL_CATALOG_PATH.read_text(encoding="utf-8"))
universes = cfg.get("universes", {})
rf1_values = set(meta["rf_level1"].dropna().astype(str).unique().tolist())

compatible = []
for uname, ucfg in universes.items():
    filt = (ucfg or {}).get("filters", {})
    rf1 = filt.get("rf_level1")
    if rf1 is None or rf1 in rf1_values:
        compatible.append(uname)

check("At least one compatible universe in config", len(compatible) > 0, detail=f"sample={compatible[:10]}")

In [ ]:
# Cell 14: Check 10 write/read smoke to parquet structure
STAGING_ROOT.mkdir(parents=True, exist_ok=True)
ts_out = STAGING_ROOT / "raw" / "timeseries_raw"
rf_out = STAGING_ROOT / "raw" / "risk_factors"
ts_out.mkdir(parents=True, exist_ok=True)
rf_out.mkdir(parents=True, exist_ok=True)

for bd, part in ts.groupby(ts["business_date"].dt.strftime("%Y-%m-%d"), sort=True):
    p = ts_out / f"business_date={bd}"
    p.mkdir(parents=True, exist_ok=True)
    part.drop(columns=["business_date"]).to_parquet(p / "part-000.parquet", index=False)

meta.to_parquet(rf_out / "risk_factors.parquet", index=False)

# smoke read
ts_smoke = pd.read_parquet(ts_out)
rf_smoke = pd.read_parquet(rf_out / "risk_factors.parquet")
check("Parquet write/read smoke - timeseries_raw", len(ts_smoke) > 0, detail=f"rows={len(ts_smoke):,}")
check("Parquet write/read smoke - risk_factors", len(rf_smoke) > 0, detail=f"rows={len(rf_smoke):,}")

In [ ]:
# Cell 15: Final report + next commands
report = pd.DataFrame(results)
display(report)

hard_fail = report[(report["ok"] == False) & (report["severity"].isin(["ERROR"]))]

if not hard_fail.empty:
    raise RuntimeError(f"Validation failed with {len(hard_fail)} ERROR checks. Fix before ingestion.")

print("Validation passed (no ERROR checks).")
print("\nNext step run example:")
print(rf"""
python -m pipeline.run_demo_layer5_runEngine ^
  --no-generate-demo-data ^
  --raw-path {STAGING_ROOT / 'raw'} ^
  --processed-path {STAGING_ROOT / 'processed'} ^
  --config-path configs/model_catalog.yaml ^
  --universe-name CP_RATE_CORP ^
  --start-date 2007-01-01 ^
  --end-date {TARGET_BUSINESS_DATE} ^
  --business-date {TARGET_BUSINESS_DATE}
""")